# ETl para sistema de recomendación de películas basado en el contenido y desarrollo de APIs

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from ast import literal_eval

In [16]:
#DATOS

df_mov = pd.read_csv(r"dsets\movies_dataset.csv", parse_dates = ['release_date'])
df_credits = pd.read_csv(r"dsets\credits.csv")

C:\Users\romin\AppData\Local\Temp\ipykernel_24276\616045571.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mov = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\PROYECTOS\Sist_recom._movies\dsets\movies_dataset.csv", parse_dates = ['release_date'])


# ETL

El objetivo es la preparación de los datos para la implementación del sistema de recomendación y de las APIs.

Ambos sets de datos deberán unirse, se chequea la compatibilidad:

In [3]:
# búsqueda de columnas en común y registros repetidos o faltantes

print(df_mov.columns)
print(df_credits.columns)

print('\nFilas de películas',len(df_mov))
print('Filas de créditos',len(df_credits))

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Index(['cast', 'crew', 'id'], dtype='object')

Filas de películas 45466
Filas de créditos 45476


In [4]:
# corroborar que hay registros duplicados para eliminarlos

len(df_credits[df_credits['id'].duplicated()])

44

In [17]:
df_credits = df_credits.drop_duplicates(subset='id', keep='first')


Join

In [6]:
# chequeo compatibilidad del tipo de datos columna "ID"

print(type(df_credits['id'][0]))
print(type(df_mov['id'][0]))

<class 'numpy.int64'>
<class 'str'>


In [18]:
# solucionar incompatibilidad de tipo de datos

df_credits['id'] = df_credits['id'].astype(str)

In [19]:
# JOIN. Uso outer sobre movies (45466), porque tiene mas registros que credits (45432). 

df_movies=df_mov.merge(df_credits, how = 'outer', on = 'id')

In [10]:
# registros que quedaron vacíos en Cast o Crew

sin_cast = df_movies[df_movies['cast'].isnull()]
sin_cast

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
19748,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29512,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35594,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42886,False,NaN,0,[],NaN,401840,tt3291632,es,School's out,Two high school kids mentored by a nightclub o...,...,0.0,[],Released,NaN,School's out,False,0.0,0.0,NaN,NaN


In [20]:
# se eliminan filas que se encontraron con problemas en el registro original

df_movies = df_movies.dropna(subset=['cast'])


# Transformaciones

In [21]:
# tratamiento de nulos en columnas sobre las que se harán operaciones

df_movies['budget'] = df_movies['budget'].fillna(0)
df_movies['revenue'] = df_movies['revenue'].fillna(0)

In [22]:
#también se eliminan nulos en Cast y Crew porque pueden interferir al aplanar los diccionarios
df_movies = df_movies.dropna(subset=['cast'])
df_movies = df_movies.dropna(subset=['crew'])

In [23]:
# Tratamiento de las fechas. 
# Extracción del año y creación de columna 'Release year'
# se separa la columna de la fecha en 3 y se eliminan las que sobran

df_movies[['release_year','release_month', 'release_day']] = df_movies.release_date.str.split(pat='-',expand=True)

del df_movies['release_month']
del df_movies['release_day']

# pasar a formato fecha aaa/mm/dd. Los datos que no son strings se convierten en nulos para luego filtrarlos y eliminarlos
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format="%Y/%m/%d", errors = 'coerce')

df_movies = df_movies.dropna(subset=['release_date'])


In [24]:
# se crea la columna con el retorno de inversión, llamada 'return' con los campos revenue y budget (revenue/budget). Si no hay datos disponibles para calcularlo, deberá tomar el valor 0.
def calcularRetorno(row):
    row['revenue'] = pd.to_numeric(row['revenue'], errors = 'coerce')
    row['budget'] = pd.to_numeric(row['budget'], errors = 'coerce')
    if row['revenue'] == 0 or row['budget'] == 0:
        return 0
    else:
        return round(row['revenue'] / row['budget'],2)

df_movies['return'] = np.where((df_movies['revenue'] == 0) | (df_movies['budget'] == 0), 0, df_movies.apply(calcularRetorno, axis=1))


In [25]:
# Se eliminan columnas que no se van a usar para el modelo ni para las APIs
columnas_aEliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df_movies.drop(columns = columnas_aEliminar, inplace=True)


In [26]:
# desanidado de columnas
def desanidar(obj):
    lista = []
    for i in literal_eval (obj):
        try:
            lista.append(i['name'])
        except (ValueError, TypeError, KeyError):
            pass
    return lista

columnas_aDesanidar = ['genres', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
df_movies[columnas_aDesanidar] = df_movies[columnas_aDesanidar].applymap(desanidar)


In [27]:
import ast
def desanidar(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [28]:
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(desanidar)

In [29]:
def desanidar(obj):
    lista = []
    for i in literal_eval (obj):
        if i['job'] == 'Director':
            try:
                lista.append(i['name'])
            except (ValueError, TypeError, KeyError):
                pass
    return lista

In [30]:
df_movies['crew'] = df_movies['crew'].apply(desanidar)

In [ ]:
# se exporta csv completo limpio para consumir
df_movies.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv')
